In [89]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

#ipython add-ons
from IPython.parallel import Client
from IPython.display import Image
import multiprocessing

##nipype
import nibabel as nib
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import BET
from nipype.interfaces.freesurfer.preprocess import ReconAll
from nipype.interfaces.freesurfer.utils import MakeAverageSubject
from nipype.interfaces.fsl import ExtractROI
from nipype.interfaces.fsl import Merge
from nipype.interfaces.fsl import TOPUP
from nipype.interfaces.fsl import ApplyTOPUP

%matplotlib inline

In [95]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'scripts/sub_cb_mappings.txt'
acq_params = home_dir + 'scripts/acqparams.txt'
os.chdir(home_dir)

num_slices = 57 #if you don't know, use nib.load and shape attribute
#get subject list
sub_list = []
with open(subj_file) as f:
    for line in f:
        (key,val) = line.strip().split(' ') 
        sub_list.append('fd_' + key)

In [35]:
def add_slice(sub):
    import os
    import glob
    from nipype.interfaces.fsl import ExtractROI
    scans = glob.glob('data/' + sub + '/cal/*') #calibration scans
    scans.extend(glob.glob('data/' + sub + '/func/*/*.nii.gz')) #functional scans
    scans = [s for s in scans if len(s.split('/')[-1].split('_'))==2] #just scans that havent been run
    
    for scan in scans:
        old_file = abspath(scan)
        new_file = abspath(scan[:-7] + '_' + str(num_slices + 1)) + '.nii.gz'
        if not os.path.exists(new_file): #only run if target doesnt exist. 
            fslroi = ExtractROI(in_file=old_file, roi_file=new_file,
                                x_min=0,x_size=-1,y_min=0,y_size=-1,
                                z_min=0,z_size= (num_slices + 1))
            fslroi.run()
    return os.getpid()

In [34]:
##add a z slice to all scans

#delete old versions. Uncomment if re-running and debugging
# for sub in sub_list:
#     scans = glob.glob('data/' + sub + '/cal/*58*') #calibration scans
#     scans.extend(glob.glob('data/' + sub + '/func/*/*58*.nii.gz')) #functional scans    
#     for scan in scans:
#         os.remove(scan)

pool = multiprocessing.Pool(processes = 5)
pool.map(add_slice,sub_list)

[9126,
 9128,
 9133,
 9132,
 9130,
 9126,
 9132,
 9126,
 9130,
 9128,
 9126,
 9133,
 9126,
 9126,
 9132]

In [52]:
##take a timepoint of the gradient reversal scans
def slice_scans(sub):
    identifier = str(num_slices + 1) #file identifier
    scans = glob.glob('data/' + sub + '/cal/*' + identifier + '*') #padded calibration scans
    for scan in scans:
        old_file = abspath(scan)
        new_file = abspath(scan[:-7] + '_slice.nii.gz')
        fslroi = ExtractROI(in_file=old_file, roi_file=new_file,
                            t_min=1,t_size=1)
        fslroi.run()
    return

In [53]:
#exectute. Delete old files first if debugging
# for sub in sub_list:
#     scans = glob.glob('data/' + sub + '/cal/*slice*') #calibration scans
#     for scan in scans:
#         os.remove(scan)
pool = multiprocessing.Pool()
pool.map(slice_scans,sub_list)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [82]:
##merge the calibration scans
def merge_scans(sub):
    identifier = 'slice' #file identifier
    cal1 = glob.glob('data/' + sub + '/cal/cal1*' + identifier + '*') #padded calibration scans
    cal2 = glob.glob('data/' + sub + '/cal/cal2*' + identifier + '*') #padded calibration scans

    if sub == 'fd_115': #extra set of scans for this subject (see notes)
        num_scans = 5
    else:
        num_scans = 4
    
    for i in range(1,num_scans+1):
        pe1 = [s for s in cal1 if int(s.split('/')[-1].split('_')[1]) == i][0]
        pe0 = [s for s in cal2 if int(s.split('/')[-1].split('_')[1]) == i][0]
        out = abspath('data/' + sub + '/cal/b0_both_' + str(i) + '.nii.gz')
        merger = Merge(in_files=[pe0,pe1], merged_file=out,dimension = 't')
        merger.run()
    return


In [83]:
pool = multiprocessing.Pool()
pool.map(merge_scans,sub_list)

INFO:interface:stderr 2015-04-29T21:59:52.340143:
INFO:interface:stderr 2015-04-29T21:59:52.340143:WARNING:: Inconsistent orientations for individual images when attempting to merge.
INFO:interface:stderr 2015-04-29T21:59:52.340143:          Merge will use voxel-based orientation which is probably incorrect - *PLEASE CHECK*!
INFO:interface:stderr 2015-04-29T21:59:52.340143:


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [99]:
#execute topup
def topup_scans(sub):
    scans = glob.glob('data/' + sub + '/cal/b0*') #padded calibration scans
    for scan in scans:
        new_file = abspath(scan[:-7] + '_topup')   
        topup = TOPUP(in_file=scan, encoding_file = acq_params, out_base = new_file)
        topup.run()
    return

In [ ]:
pool = multiprocessing.Pool()
pool.map(topup_scans,sub_list)

In [ ]:
##apply topup to functional runs
def apply_topup(sub):
    #padded functional scans
    scans = glob.glob('data/' + sub + '/func/*/*' + str(num_slices + 1) + '.nii.gz') 
    
    for scan in scans:
        if scan

    
    return


In [ ]:
ApplyTopup.help()